In [36]:
import numpy as np
import nltk
import string
import string
import re
import pdfminer.high_level
from pdfminer.high_level import extract_pages, extract_text
import random

In [6]:
pip install pdfminer.six

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 71.3 MB/s eta 0:00:00


In [2]:
pip install pdfminer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 54.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.4 MB/s eta 0:00:00
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140107 sha256=7ce8bd56cc70cb29bc1282a3bb0ee29a3932d0d16494d0e61be8ceaf62e88848
  Stored in directory: /root/.cache/pip/wheels/1c/28/7d/f390b82bb0307deb63ff27a1474fd308ec68ee028cb9ab6283
Successfully built pdfminer


In [48]:
txt = extract_text("ra.pdf")
txt

'Routing Algorithms\n\n\x0c\uf09e Introduction\n\uf09e Distance-Vector Routing\n\n- Bellman-Ford Equation.\n- Distance Vectors.\n- Distance-Vector Routing Algorithm.\n\n\uf09e Link-State Routing\n\n- Link-State Database (LSDB).\n- Formation of Least-Cost Trees.\n\n\uf09e Path-Vector Routing\n- Spanning Trees.\n- Creation of Spanning Trees.\n- Path-Vector Algorithm.\n\n\x0cAn internet is a combination of networks connected\nby routers. When a datagram goes from a source to\na destination, it will probably pass through many\nrouters until it reaches the router attached to the\ndestination network.\n\n\x0cToday, an internet can be so large that one \nrouting protocol cannot handle the task of \nupdating the routing tables of all routers. For \nthis reason, an internet is divided into \nautonomous systems. \nAn autonomous system (AS) is a group of \nnetworks and routers under the authority of a \nsingle administration. Routing inside an \nautonomous system is called intra-domain \nrouting.

In [ ]:
pattern = re.compile(r"[a-zA-A]+,{1}\s{1}")
matches = pattern.findall(txt)
matches

In [12]:

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [49]:
sentence_tokens = nltk.sent_tokenize(txt)
word_tokens = nltk.word_tokenize(txt)

In [50]:
sentence_tokens[5]

'\uf09e Path-Vector Routing\n- Spanning Trees.'

In [51]:
word_tokens[:10]

['Routing',
 'Algorithms',
 '\uf09e',
 'Introduction',
 '\uf09e',
 'Distance-Vector',
 'Routing',
 '-',
 'Bellman-Ford',
 'Equation']

In [52]:
lemmer = nltk.stem.WordNetLemmatizer()
def lemtokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]
remove_punc_dict = dict((ord(punct),None) for punct in string.punctuation)
def lemnormalize(text):
  return lemtokens(nltk.word_tokenize(text.lower().translate(remove_punc_dict)))

In [53]:
greet_in = ('hello','hey','hi','wassup','how u doin','how are you?')
greet_res = ('hey','hi','Hey there!','There there!')
def greet(sentence):
  for word in sentence.split():
    if word.lower() in greet_in:
      return random.choice(greet_res)

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [55]:
def response(user_res):
  rob_res = ''
  TfidVec = TfidfVectorizer(tokenizer = lemnormalize,stop_words='english')
  tfidf = TfidVec.fit_transform(sentence_tokens)
  vals = cosine_similarity(tfidf[-1],tfidf)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]
  if req_tfidf==0:
    rob_res += "I am sorry. Unable to understand you."
    return rob_res
  else:
    for i in range(25):
      rob_res = rob_res + sentence_tokens[idx+i]
    return rob_res

In [56]:
flag = True
print("Hello there! Ask your curious question please.Also provide the number of sentences you want")
while(flag==True):

  user_res = input("You: " )
  #user_n = int(input("No of sentences: "))
  user_res = user_res.lower()
  if(user_res!='bye'):
    if(user_res == 'thank you' or user_res == 'thanks'):
      flag = False
      print("Bot: You are welcome...")
    else:
      if(greet(user_res)!= None):
        print("Bot "+greet(user_res))
      else:
        sentence_tokens.append(user_res)
        word_tokens+=nltk.word_tokenize(user_res)
        final_words = list(set(word_tokens))
        print("Bot: "+response(user_res))
        sentence_tokens.remove(user_res)
  else:
    flag =False
    print("Bot: Goodbye!!")

Hello there! Ask your curious question please.Also provide the number of sentences you want
You: link state routing
Bot: -The cost of the link. The combination of these two pieces of information is called 

the LS packet (LSP). The LSP is sent out of each interface, as shown in Figure 20.9 

for our internet in Figure 20. When a node receives an LSP from one of its interfaces, it 

compares the LSP with the copy it may already have. If the arrived LSP is older than the one it has, it discards 

the arrived LSP. If it is newer than the one it has, the node discards the old 

LSP and keeps the received one. It then sends a copy of it out of each interface except the 

one from which the packet arrived.To create a least-cost tree for itself, using the shared LSDB, 
each node needs to run the famous Dijkstra Algorithm.This iterative algorithm uses the following steps:
1.The node chooses itself as the root of the tree, creating 
a tree with a single node, and sets the total cost of ea